In [19]:
import pandas as pd
import numpy as np
import pickle
from pandas import ExcelWriter
import ffn
%matplotlib inline

def save_xls(list_dfs, xls_path,sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return

#pull in data from AlphaVantage
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='NXY0VT9AHBRYGKKC',output_format='pandas')



In [20]:
active_stocks = pd.read_csv('active_names.csv', index_col = 0).dropna()

In [21]:
def historical_data(ticker, outsize = "full"):
    #outsize can be compact and full. 
    #compact returns only the latest 100 data points; full 
    #returns the full-length time series of up to 20 years of historical data. The "compact" option is 
    #recommended if you would like to reduce the data size of each API call
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={0}&apikey=NXY0VT9AHBRYGKKC&datatype=csv&outputsize={1}'.format(ticker, outsize)
    df = pd.read_csv(alphavantage_link)
    df.index = pd.to_datetime(df.timestamp)
    df = df[['adjusted_close']]
    df.columns = [ticker]
    df.dropna(inplace = True)
    return df

def multiple_tickers(tickers, df):
    for ticker in tickers:
        temp_df = historical_data(ticker)
        df = df.join(temp_df, how = "left")
        df.dropna(inplace = True)
        df = df.sort_index()
    df = df.loc[~(df==0).all(axis=1)]
    return df

In [22]:
MOMO = active_stocks.index[0:501]

In [23]:
MAIN = historical_data("SPY")

In [28]:
L =[ticker for ticker in MOMO]

In [ ]:
Main_DF = multiple_tickers(L,MAIN)

In [ ]:
with open('filename.pickle', 'wb') as handle:
    pickle.dump(Main_DF, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('filename.pickle', 'rb') as handle:
    b = pickle.load(handle)